In [1]:
import os
import sys
import scanpy as sc
import pandas as pd
sys.path.append('..')
from constants import BASE_PATH_DATA

In [2]:
adata = sc.read_h5ad(os.path.join(BASE_PATH_DATA, 'cansig_processed/breast_large.h5ad'))

In [3]:
adata

AnnData object with n_obs × n_vars = 68961 × 14378
    obs: 'Patient', 'Percent_mito', 'nCount_RNA', 'nFeature_RNA', 'celltype', 'celltype_minor', 'celltype_subset', 'subtype', 'gene_module', 'Calls', 'normal_cell_call', 'CNA_value', 'sample_id', 'n_counts', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'malignant_celltype', 'reference', 'malignant_cnv', 'malignant_key', 'subclonal', 'S_score', 'G2M_score', 'phase', 'Basal_SC_GT', 'Her2E_SC_GT', 'LumA_SC_GT', 'LumB_SC_GT', 'GM1_GT', 'GM2_GT', 'GM3_GT', 'GM4_GT', 'GM5_GT', 'GM6_GT', 'GM7_GT', 'log_counts'
    var: 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'n_cells', 'chromosome', 'start', 'end', 'cnv_called'
    uns: 'cnv'
    obsm: 'X_cnv'

In [4]:
adata.obs.sample_id.value_counts().sort_index()

CID3921     2698
CID3941      497
CID3948      928
CID3963     3210
CID4066     3880
CID4067     2853
CID4290A    4792
CID4461      196
CID4463      928
CID4465      350
CID4471     8327
CID4495     6950
CID4513     5317
CID4515     3473
CID4523     1336
CID4530N    4066
CID4535     3483
CID44971    7485
CID44991    6143
CID45171    2049
Name: sample_id, dtype: int64

In [5]:
adata.obs.malignant_key.value_counts().sort_index()

malignant        22409
non-malignant    46312
undecided          240
Name: malignant_key, dtype: int64

In [6]:
adata = adata[adata.obs.malignant_key == 'malignant'].copy()

In [7]:
def create_pivot_table(data, index, column):
    tmp = data.obs[[index, column]].value_counts().sort_index().reset_index()
    tmp.columns = [index, column, 'count']
    return pd.pivot(tmp,
                    columns=column,
                    index=index,
                    values='count')

In [8]:
create_pivot_table(adata,'gene_module', 'malignant_key')

malignant_key,malignant
gene_module,
1,4195
2,3068
3,2377
4,3756
5,3662
6,2400
7,2951


In [9]:
create_pivot_table(adata,
                   'celltype', 
                   'malignant_key').fillna(0)

malignant_key,malignant
celltype,
Cancer Epithelial,22409


In [14]:
adata.write(os.path.join(BASE_PATH_DATA, 'cansig_processed/breast_malignant.h5ad'))